In [140]:
import json
import pandas as pd
import numpy as np

%run Tokenization_and_stemming.ipynb import tokenize, stem, bag_of_words
%run NN_model1.ipynb import NeuralNet

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

[nltk_data] Downloading package punkt to C:\Users\D.UDAY
[nltk_data]     KIRAN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [141]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [142]:
all_words = []
tags = []
xy = []

In [143]:
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))
        
ignore_words = ['?',',','!','.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [144]:
x_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence,all_words)
    x_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label)
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [145]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train
    
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.n_samples        

In [146]:
#hyperparameters
batch_size = 8
hidden_size = 8
ouput_size = len(tags)
input_size = len(x_train[0])
learning_rate = 0.001
num_epochs = 1000

In [147]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size,hidden_size,ouput_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [148]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        #forward
        outputs = model(words)
        loss = criterion(outputs,labels)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print(f'epoch {epoch+1}/{num_epochs}, loss = {loss.item():.4f}')
        
print(f'final loss, loss = {loss.item():.4f}')

epoch 100/1000, loss = 1.0040
epoch 200/1000, loss = 0.5386
epoch 300/1000, loss = 0.1503
epoch 400/1000, loss = 0.0109
epoch 500/1000, loss = 0.0085
epoch 600/1000, loss = 0.0007
epoch 700/1000, loss = 0.0014
epoch 800/1000, loss = 0.0015
epoch 900/1000, loss = 0.0023
epoch 1000/1000, loss = 0.0003
final loss, loss = 0.0003
